In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from load_paus_cat import load_paus_cat
from paus_utils import PAUS_monochromatic_Mag

In [2]:
selection = pd.read_csv('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection.csv')
selection.dtypes

Unnamed: 0               int64
ref_id                   int64
field                   object
RA                     float64
DEC                    float64
r_mag                  float64
lya_NB                   int64
EW0_lya                float64
EW0_lya_err            float64
L_lya                  float64
L_lya_corr             float64
class_pred               int64
z_NB                   float64
class_star             float64
nice_lya                  bool
nice_ml                   bool
nice_color                bool
M_UV                   float64
M_UV_err               float64
L_lya_corr_err_up      float64
L_lya_corr_err_down    float64
dtype: object

In [3]:
from jpasLAEs.utils import flux_to_mag
from paus_utils import NB_z
def PAUS_monochromatic_Mag(flx, err, redshift, wavelength=1450):
    '''
    Calculate the absolute magnitude (M) and its error for sources in a catalog
    at a specified monochromatic wavelength in the rest-frame.

    Parameters:
    cat (dict): A dictionary containing information about sources, including:
        - 'z_NB' (numpy.ndarray): Redshift values for sources.
        - 'flx' (numpy.ndarray): Flux values for sources.
        - 'err' (numpy.ndarray): Flux error values for sources.
        - 'nice_lya' (numpy.ndarray): Boolean mask for selecting sources.

    wavelength (float, optional): The desired wavelength in Angstroms in the rest-frame.
        Default is 1450 Angstroms.

    Returns:
    tuple: A tuple containing two numpy arrays:
        - M_Arr (numpy.ndarray): Absolute magnitude for sources. Sources without valid
          data are assigned a value of 99.
        - magAB_err_Arr (numpy.ndarray): Error in the absolute magnitude.
    '''

    # Find the NB of the specified wavelength in rest-frame
    nb_w_rest = NB_z(redshift, wavelength)
    dist_lum_Arr = cosmo.luminosity_distance(redshift).to(u.pc).value
    dist_lum_err_Arr = cosmo.luminosity_distance(redshift + 0.06).to(u.pc).value - dist_lum_Arr

    N_sources = len(redshift)

    flambda_Arr = np.ones(N_sources).astype(float) * 99.
    flambda_err_Arr = np.copy(flambda_Arr)

    src_list = np.arange(len(redshift))
    for src in src_list:
        nb_min = nb_w_rest[src] - 3
        if nb_min == -1:
            nb_min = 38
        nb_max = nb_min + 2
        if nb_max == nb_min:
            continue

        w = err[nb_min : nb_max, src] ** -2
        flambda_Arr[src] = np.average(flx[nb_min : nb_max, src],)
                                    #   weights=w)
        flambda_err_Arr[src] = np.sum(w, axis=0) ** -0.5

    magAB_Arr = flux_to_mag(flambda_Arr, wavelength * (1 + redshift))
    magAB_err_Arr = magAB_Arr - flux_to_mag(flambda_Arr + flambda_err_Arr,
                                             wavelength * (1 + redshift))

    M_Arr = np.ones(N_sources) * 99
    M_Arr_err = np.ones(N_sources) * 99

    M_Arr = magAB_Arr - 5 * (np.log10(dist_lum_Arr) - 1)

    M_Arr_err = (magAB_err_Arr ** 2 + (5 / dist_lum_Arr * np.log10(np.e) * dist_lum_err_Arr) ** 2) ** 0.5

    return M_Arr, M_Arr_err

In [ ]:
from paus_utils import w_central

sel_coords = SkyCoord(selection['RA'], selection['DEC'], unit='deg')

M_UV = np.empty(len(selection))
r_paus = np.empty(len(selection))

for field_name in ['W1', 'W2', 'W3']:
    path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
    cat = load_paus_cat(path_to_cat)

    this_cat_coords = SkyCoord(cat['RA'], cat['DEC'], unit='deg')

    this_ids, this_dist, _ = sel_coords.match_to_catalog_sky(this_cat_coords)

    mask_dist = this_dist.value < (3 / 3600)

    cat['z_NB'] = np.zeros_like(cat['ref_id'])
    redshift_arr = np.array(selection['z_NB'][mask_dist])
    print(redshift_arr)

    r_paus[mask_dist] = flux_to_mag(cat['flx'][-4][this_ids][mask_dist], w_central[-4])

    M_UV[mask_dist], _ = PAUS_monochromatic_Mag(cat['flx'].T[this_ids][mask_dist].T,
                                             cat['err'].T[this_ids][mask_dist].T,
                                             redshift_arr)
                                            
print(M_UV)

In [ ]:
# SDSS catalog
path_to_cat = '/home/alberto/almacen/SDSS_spectra_fits/DR18/spAll-v5_13_2.fits'
sdss_cat = fits.open(path_to_cat)[1].data

In [ ]:
# Do the cross-matches
coords_paus = SkyCoord(ra=np.asarray(selection['RA']) * u.deg,
                       dec=np.asarray(selection['DEC']) * u.deg)
coords_sdss = SkyCoord(ra=sdss_cat['RA'] * u.deg,
                       dec=sdss_cat['DEC'] * u.deg)

xm_id_sdss, ang_dist_sdss, _= coords_paus.match_to_catalog_sky(coords_sdss)

# Objects with 1 arcsec of separation
mask_dist_sdss = (ang_dist_sdss <= 1.5 * u.arcsec) & (sdss_cat['ZWARNING'][xm_id_sdss] == 0)

mask_dist_sdss = np.asarray(mask_dist_sdss)

In [ ]:
from jpasLAEs.utils import mag_to_flux

def nanomaggie_to_mag(nmagg):
    mAB = -2.5 * np.log10(nmagg * 1e-9)
    return mAB

def nanomaggie_to_flux(nmagg, wavelength):
    mAB = -2.5 * np.log10(nmagg * 1e-9)
    flx = mag_to_flux(mAB, wavelength)
    return flx

fits_dir = '/home/alberto/almacen/SDSS_spectra_fits/DR16/QSO'

MUV_spec_Arr = np.ones(sum(mask_dist_sdss)) * 99
r_spec_Arr = np.ones(sum(mask_dist_sdss)) * 99
r_band_sdss = np.ones(sum(mask_dist_sdss)) * 99
z_sdss = np.ones(sum(mask_dist_sdss)) * 99

for src in range(sum(mask_dist_sdss)):
    plate = sdss_cat['PLATE'][xm_id_sdss][mask_dist_sdss][src]
    mjd = sdss_cat['MJD'][xm_id_sdss][mask_dist_sdss][src]
    fiber = sdss_cat['FIBERID'][xm_id_sdss][mask_dist_sdss][src]

    spec_name = f'spec-{plate:04d}-{mjd:05d}-{fiber:04d}.fits'
    try:
        spec_sdss = Table.read(f'{fits_dir}/{spec_name}', hdu=1, format='fits')
        sdss_bbs = Table.read(f'{fits_dir}/{spec_name}', hdu=2, format='fits')['SPECTROSYNFLUX']
    except:
        print(f'{spec_name} not found.')
        continue

    r_band_sdss[src] = nanomaggie_to_mag(np.array(sdss_bbs)[0][2])

    redshift = sdss_cat['Z'][xm_id_sdss][mask_dist_sdss][src]

    spec_w_rest = 10**spec_sdss['LOGLAM'] / (1 + redshift)
    spec_w_obs = 10**spec_sdss['LOGLAM']
    spec_f = spec_sdss['FLUX']

    mask_1450_w_obs = (spec_w_rest > 1400) & (spec_w_rest < 1500)
    mask_r = (spec_w_obs > 5750) & (spec_w_obs < 7000)


    if np.sum(spec_sdss['IVAR'][mask_1450_w_obs]**-1) == 0:
        continue
    m_UV_nanomagg = np.average(spec_f[mask_1450_w_obs],
                               weights=spec_sdss['IVAR'][mask_1450_w_obs]**-1)
    m_UV = nanomaggie_to_mag(m_UV_nanomagg)

    r_nanomagg = np.mean(spec_f[mask_r])
    r = nanomaggie_to_mag(r_nanomagg)

    r_spec_Arr[src] = r

    this_M_UV = m_UV - cosmo.distmod(redshift).value

    MUV_spec_Arr[src] = this_M_UV

    z_sdss[src] = redshift

In [ ]:
from jpasLAEs.utils import bin_centers
def compute_L_Lbin_err(L_binning, L_phot, L_spec):
    '''
    Computes the errors due to dispersion of L_retrieved with some L_retrieved binning
    '''

    L_Lbin_err_plus = np.ones(len(L_binning) - 1) * np.inf
    L_Lbin_err_minus = np.ones(len(L_binning) - 1) * np.inf
    median = np.ones(len(L_binning) - 1) * np.inf
    last = [np.inf, np.inf]
    for i in range(len(L_binning) - 1):
        in_bin = (L_phot >= L_binning[i]
                  ) & (L_phot < L_binning[i + 1])
        if sum(in_bin) == 0:
            L_Lbin_err_plus[i] = last[0]
            L_Lbin_err_minus[i] = last[1]
            continue
        perc = np.nanpercentile((L_phot - L_spec)[in_bin], [16, 50, 84])
        L_Lbin_err_plus[i] = perc[2] - perc[1]
        L_Lbin_err_minus[i] = perc[1] - perc[0]

        last = [L_Lbin_err_plus[i], L_Lbin_err_minus[i]]
        median[i] = perc[1]

    return L_Lbin_err_plus, L_Lbin_err_minus, median

#####

M_UV_bins = np.linspace(-30, -21, 10)

M_err_plus, M_err_minus, M_bias = compute_L_Lbin_err(M_UV_bins,
                                                       M_UV[mask_dist_sdss],
                                                       MUV_spec_Arr - (r_spec_Arr - r_band_sdss))

#####


fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(M_UV[mask_dist_sdss], MUV_spec_Arr - (r_spec_Arr - r_band_sdss))

ax.plot([-50, 50], [-50, 50], ls='--', c='r', lw=2, zorder=-99)

M_xx = bin_centers(M_UV_bins)
ax.plot(M_xx, M_xx - M_bias,
        c='peru', lw=2)
ax.plot(M_xx, M_xx + M_err_plus,
        c='peru', lw=2)
ax.plot(M_xx, M_xx - M_err_minus,
        c='peru', lw=2)


ax.set_xlabel(r'$M_{\rm UV}$ phot')
ax.set_ylabel(r'$M_{\rm UV}$ spec')

ax.set_ylim(-22, -30)
ax.set_xlim(-22, -30)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# ax.scatter(r_spec_Arr, r_band_sdss)
# ax.scatter(r_band_sdss, selection['r_mag'][mask_dist_sdss])
# ax.scatter(r_paus[mask_dist_sdss], selection['r_mag'][mask_dist_sdss])
ax.scatter(r_band_sdss, r_paus[mask_dist_sdss])

ax.set_xlabel(r'$r_{\rm SDSS}$')
ax.set_ylabel(r'$r_{\rm PAUS}$')

ax.plot([-100, 100], [-100, 100], c='k', lw=2)

ax.set_ylim(18, 25)
ax.set_xlim(18, 25)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

to_bias = M_UV[mask_dist_sdss] - (MUV_spec_Arr - (r_spec_Arr - r_band_sdss))
bias = np.nanmean(to_bias[MUV_spec_Arr < 30])
std = np.nanstd(to_bias[MUV_spec_Arr < 30])
print(np.nanpercentile(to_bias[MUV_spec_Arr < 30], [16, 84]))
print(bias)
print(std)
ax.hist(M_UV[mask_dist_sdss] - (MUV_spec_Arr - (r_spec_Arr - r_band_sdss)),
        np.linspace(-3, 3, 20), histtype='step')
ax.axvline(bias, ls='--', c='red', lw=2)
ax.axvline(bias + std, ls='--', c='red', lw=2)
ax.axvline(bias - std, ls='--', c='red', lw=2)

ax.set_xlabel(r'$\Delta M_{\rm UV}$ (phot - spec)')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(selection['r_mag'][mask_dist_sdss], r_band_sdss)
ax.scatter(r_spec_Arr, r_band_sdss)

ax.plot([-50, 50], [-50, 50], ls='--', c='r', lw=2, zorder=-99)

ax.set_xlabel(r'$M_{\rm UV}$ phot')
ax.set_ylabel(r'$M_{\rm UV}$ spec')

ax.set_ylim(15, 25)
ax.set_xlim(15, 25)

plt.show()